Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

In [ ]:
# Install required packages
!pip install -q rdflib pydot networkx pyvis     # comment to avoid re-install with every re-run
     

In [ ]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace

# Task 1 ( 1 pt)

Sie beginnen mit einem leeren Dataset. Fügen Sie in den Default Graph Statements ein, die sagen, dass :Peter der Autor von :G1 ist, und :Mary Autor von :G2.

In [ ]:
ds = rdflib.Dataset()

ds.parse(format="trig", data="""
@prefix :    <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  .
""")

ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT DATA { GRAPH <urn:x-rdflib:default>  {
:G1  :author  :Mary .
:G2  :author  :Peter .
}}
""")

print(ds.serialize(format="trig"))


# Task 2 ( 1 pt)

Schreiben Sie { :Mary :knows :Peter, :John, :Mary. } in den Named Graph :G1 und { :Peter :knows :Mary. :John :knows :Mary. } in den Named Graph :G2.



In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT DATA {
GRAPH :G1{
 :Mary :knows :Peter, :John, :Mary. 
 }

GRAPH :G2{
 :Peter :knows :Mary. :John :knows :Mary.
 }

}
""")

print(ds.serialize(format="trig"))

# Task 3 ( 1 pt)

Fragen Sie mittels INSERT-WHERE die :knows-Beziehungen aus :G2 ab und fügen deren inverse :knownBy-Beziehungen in den Default-Graph ein.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]


In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT {GRAPH <urn:x-rdflib:default> {?o :knownBy ?s}}
WHERE
{
GRAPH :G2
{?s :knows ?o}
}

""")

print(ds.serialize(format="trig"))

# Task 4 ( 1 pt)

Löschen Sie mittels DELETE-WHERE alle :knownBy-Beziehungen aus dem Default-Graph.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE WHERE{ GRAPH <urn:x-rdflib:default>  {?s :knownBy ?o}}
""")

print(ds.serialize(format="trig"))

# Task 5 ( 1 pt )

Ermitteln Sie mittels INSERT-WHERE zu jedem Named-Graph dessen Anzahl an Statements mit der Property :knows und schreiben Sie diese in den Default Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]



In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT { GRAPH <urn:x-rdflib:default> {?g :knowsCount ?nr }}
WHERE {
 SELECT ?g (COUNT(*) as ?nr)
 WHERE {
 GRAPH ?g {?s :knows ?o }
 } GROUP BY ?g
}
""")

print(ds.serialize(format="trig"))

# Task 6 ( 1 pt )

Ermitteln Sie mittels INSERT-WHERE die Anzahl an Named Graphs und schreiben Sie sie in den Default Graph.
Ihr Update Request darf nicht enthalten: [2]




In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

INSERT { GRAPH <urn:x-rdflib:default> {:ds :graphCount ?nr}}
WHERE {
        SELECT (COUNT (DISTINCT ?g) as ?nr) 
        WHERE {
        GRAPH ?g{?s ?p ?o}
        } 
}
""")

print(ds.serialize(format="trig"))

# Task 7 ( 1pt )
Verschieben Sie mittels DELETE-INSERT-WHERE alle Metadaten zu Named Graphs (also Statements die einen Named Graph als Subjekt haben) in den entsprechenden Named Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]

In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE {?s ?p ?o}
INSERT {
  GRAPH ?s {
    ?s ?p ?o
  }
}
WHERE {
  ?s ?p ?o
  FILTER(?s != :ds)
}
""")

print(ds.serialize(format="trig"))

# Task 8 ( 1 pt )

Schreiben Sie in jeden Named Graph ein Statement, dass der Autor des jeweiligen Named Graphs die :Susi kennt und aktualisieren Sie mit dem selben UpdateRequest den knowsCount.
Ihr Update Request darf nicht enthalten: [Peter, Mary]

In [ ]:
ds.update("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE {GRAPH ?g {?g :knowsCount ?c}}
INSERT {
  GRAPH ?g {
    ?o :knows :Susi.
    ?g :knowsCount ?n .
  }
}
WHERE {
  GRAPH ?g { 
  ?s :author ?o.
  ?g :knowsCount ?c .
  BIND(?c + 1 AS ?n)
  }
}
""")

print(ds.serialize(format="trig"))